In [4]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Load training and test data
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

# Convert relative paths to absolute paths
df_train["Path"] = df_train["Path"].apply(os.path.abspath)
df_test["Path"] = df_test["Path"].apply(os.path.abspath)

# Encode ClassId numerically
encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])

# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=50)  # Increased number of features

# Store descriptors for each class
class_descriptors = {i: [] for i in df_train["ClassId"].unique()}

def extract_orb_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    return orb.detectAndCompute(img, None)[1]  # Return descriptors only

# Compute ORB descriptors for training images
for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Extracting ORB Features"):
    descriptors = extract_orb_features(row.Path)
    if descriptors is not None:
        class_descriptors[row.ClassId].append(descriptors)

# Function to match descriptors using FLANN matcher
def match_features(test_desc, train_desc_list):
    if test_desc is None or len(test_desc) == 0:
        return 0  # No descriptors means no matches

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    best_matches = 0

    for train_desc in train_desc_list:
        if train_desc is None or len(train_desc) == 0:
            continue  # Skip invalid descriptors
        
        matches = matcher.knnMatch(test_desc, train_desc, k=2)
        good_matches = [m for m in matches if len(m) == 2 and m[0].distance < 0.7 * m[1].distance]  # Fix applied here
        best_matches = max(best_matches, len(good_matches))

    return best_matches

# CNN Model Definition
def create_cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Prepare data for CNN
IMG_SIZE = (224, 224)

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE) if img is not None else np.zeros((*IMG_SIZE, 3), dtype=np.uint8)
    return img / 255.0  # Normalize

# Load and preprocess training data
X_train = np.array([load_and_preprocess_image(row.Path) for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Loading Training Data")])
y_train = df_train["ClassId"].values

# Augment data to improve generalization
data_gen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and compile the CNN model
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
num_classes = len(df_train["ClassId"].unique())
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model with early stopping
cnn_model.fit(data_gen.flow(X_train, y_train, batch_size=32),
              validation_data=(X_val, y_val),
              epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=3)],
              verbose=1)

# Predict test images
predictions = []
for row in tqdm(df_test.itertuples(), total=len(df_test), desc="Matching Test Images"):
    test_desc = extract_orb_features(row.Path)
    match_scores = {class_id: match_features(test_desc, class_descriptors[class_id]) for class_id in class_descriptors}
    best_class = max(match_scores, key=match_scores.get)
    
    if match_scores[best_class] < 15:  # Increased threshold for better confidence
        img = np.expand_dims(load_and_preprocess_image(row.Path), axis=0)
        preds = cnn_model.predict(img, verbose=0)
        if np.max(preds) > 0.7:  # Ensure CNN confidence is high enough
            best_class = np.argmax(preds)
    
    predictions.append(best_class)

# Save predictions
df_submission = pd.DataFrame({"id": df_test["id"], "output": predictions})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")


Loading Training Data: 100%|██████████| 5000/5000 [00:04<00:00, 1140.24it/s]
c:\Documents\DATATHON\fungi-classification-shallow-learning\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Documents\DATATHON\fungi-classification-shallow-learning\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 354ms/step - accuracy: 0.4232 - loss: 1.4241 - val_accuracy: 0.5610 - val_loss: 0.8842
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 333ms/step - accuracy: 0.7511 - loss: 0.5192 - val_accuracy: 0.5020 - val_loss: 2.0409
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 332ms/step - accuracy: 0.8034 - loss: 0.4373 - val_accuracy: 0.5690 - val_loss: 1.4454
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 329ms/step - accuracy: 0.8190 - loss: 0.3994 - val_accuracy: 0.7140 - val_loss: 0.6611
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 336ms/step - accuracy: 0.8351 - loss: 0.3893 - val_accuracy: 0.6640 - val_loss: 1.4881
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 330ms/step - accuracy: 0.8671 - loss: 0.3344 - val_accuracy: 0.6050 - val_loss: 1.4145
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 334ms/step - accuracy: 0.8220 - loss: 0.4093 - val_accuracy: 0.5830 - val_loss: 1.9690


Matching Test Images: 100%|██████████| 1801/1801 [02:43<00:00, 11.05it/s]

Prediction complete. Results saved to submission.csv
